# Round 06 — Query + Outcome Review

**Learning focus:** CNN concepts (hierarchical feature extraction analogy)

This notebook logs the query submitted for each of the 8 functions in Round 06, the portal-returned outputs, and quick diagnostics.

> Outputs come from the capstone portal black-box and cannot be regenerated locally.

In [ ]:
import os, numpy as np, pandas as pd

ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))  # run from notebooks/
ROUND_DIR = os.path.join(ROOT, "data", "round06")
IN_FILE_CANDIDATES = ["submitted_inputs.txt", "inputs.txt"]
OUT_FILE_CANDIDATES = ["returned_outputs.txt", "outputs.txt"]

def read_first_existing(path_dir, candidates):
    for fn in candidates:
        p=os.path.join(path_dir, fn)
        if os.path.exists(p):
            return p
    raise FileNotFoundError(f"No file found in {path_dir} among {candidates}")

in_path = read_first_existing(ROUND_DIR, IN_FILE_CANDIDATES)
out_path = read_first_existing(ROUND_DIR, OUT_FILE_CANDIDATES)

inputs = [ln.strip() for ln in open(in_path).read().splitlines() if ln.strip() and not ln.startswith("#")]
outputs = [float(ln.strip()) for ln in open(out_path).read().splitlines() if ln.strip() and not ln.startswith("#")]

assert len(inputs)==8 and len(outputs)==8, f"Expected 8 functions, got {len(inputs)} inputs and {len(outputs)} outputs"

df_round = pd.DataFrame({
    "function_id": list(range(1,9)),
    "query_submitted": inputs,
    "output_returned": outputs
})
df_round


## Round summary (sorted)

In [ ]:
df_round.sort_values('output_returned', ascending=False)

## Best-so-far up to this round

In [ ]:
import glob, re

round_dirs = sorted(glob.glob(os.path.join(ROOT, "data", "round*")))
rounds=[]
for d in round_dirs:
    m=re.search(r"round(\d+)$", d.replace('\\','/'))
    if m:
        rounds.append(int(m.group(1)))
rounds=sorted(rounds)

def load_round(r):
    rd=os.path.join(ROOT, "data", f"round{r:02d}")
    ip=read_first_existing(rd, IN_FILE_CANDIDATES)
    op=read_first_existing(rd, OUT_FILE_CANDIDATES)
    ins=[ln.strip() for ln in open(ip).read().splitlines() if ln.strip() and not ln.startswith("#")]
    outs=[float(ln.strip()) for ln in open(op).read().splitlines() if ln.strip() and not ln.startswith("#")]
    return ins, outs

history=[]
for rr in rounds:
    ins, outs = load_round(rr)
    for fid in range(1,9):
        history.append({
            "round": rr,
            "function_id": fid,
            "query": ins[fid-1],
            "y": outs[fid-1]
        })
hist = pd.DataFrame(history)
hist_upto = hist[hist["round"] <= 6].copy()

best = (hist_upto.sort_values(["function_id","y"], ascending=[True,False])
                .groupby("function_id")
                .head(1)
                .sort_values("function_id"))
best.rename(columns={"round":"best_round","query":"best_query","y":"best_y"}, inplace=True)
best[["function_id","best_round","best_y","best_query"]]


## Optional: PCA visual check (diagnostic)

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

def parse_portal(s):
    return np.array([float(x) for x in s.strip().split("-")], dtype=float)

X = np.vstack([parse_portal(s) for s in df_round["query_submitted"].values])
y = df_round["output_returned"].values

plt.figure()
if X.shape[1] > 2:
    X2 = PCA(n_components=2, random_state=0).fit_transform(X)
    plt.scatter(X2[:,0], X2[:,1])
    for i,(x1,x2) in enumerate(X2):
        plt.text(x1, x2, str(i+1), fontsize=9)
    plt.xlabel("PC1"); plt.ylabel("PC2")
    plt.title("Round points projected to 2D (PCA)")
else:
    plt.scatter(X[:,0], X[:,1])
    for i,(x1,x2) in enumerate(X):
        plt.text(x1, x2, str(i+1), fontsize=9)
    plt.xlabel("x1"); plt.ylabel("x2")
    plt.title("Round points (2D)")
plt.show()
